In [8]:
import pandas as pd
import numpy as np
import re

In [9]:
import os
os.getcwd()

'/Users/ryleyun/Desktop/Github/Ryles_Practice/DACON'

In [10]:
train = pd.read_csv("../DATA/train_data.csv")
test = pd.read_csv("../DATA/test_data.csv")

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

In [3]:
%%time
def pos(x):
    try:
        text = ''
        for word, pos in mecab.pos(str(x)):
            if pos in ["NNG", "NNP", "VV", "VA", "VCP", "VCN", "MM", "MAG", "XPN", "SL", "SH"]:
                if type(re.search("\W+|[0-9]", word))!=re.Match and len(word)>1: 
                    # and len(word)!=1:
                    text+=" "+word
        return text.strip()
    
    except:
        pass

train["과제명"] = train["과제명"].apply(pos)
train["요약문_연구목표"] = train["요약문_연구목표"].apply(pos)
train["요약문_연구내용"] = train["요약문_연구내용"].apply(pos)
train["요약문_기대효과"] = train["요약문_기대효과"].apply(pos)
train["요약문_한글키워드"] = train["요약문_한글키워드"].apply(pos)

test["과제명"] = test["과제명"].apply(pos)
test["요약문_연구목표"] = test["요약문_연구목표"].apply(pos)
test["요약문_연구내용"] = test["요약문_연구내용"].apply(pos)
test["요약문_기대효과"] = test["요약문_기대효과"].apply(pos)
test["요약문_한글키워드"] = test["요약문_한글키워드"].apply(pos)

NameError: name 'train' is not defined

In [11]:
train["concat"] = train["과제명"]+" "+train["요약문_한글키워드"]

In [21]:
train["concat"]= train["concat"].fillna("None")

In [22]:
from sklearn.model_selection import train_test_split
train_data, valid_data, train_label, valid_label = train_test_split(train["concat"], train["label"], test_size=0.1, random_state=42)

In [23]:
test["concat"] = test["과제명"]+" "+test["요약문_한글키워드"]
test["concat"]= test["concat"].fillna("None")

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

text_clf = Pipeline([("vect", CountVectorizer()), ("tfidf", TfidfTransformer()), ("clf", MultinomialNB())])
text_clf = text_clf.fit(train_data, train_label)

params_clf = {"vect__ngram_range":[(1,5),(1,7),(1,9)],
              "clf__alpha":[0.01, 0.008, 0.006]}

gs_clf = GridSearchCV(text_clf, params_clf, n_jobs=5, verbose=2)
gs_clf = gs_clf.fit(train_data, train_label)

print("Best score: {}".format(gs_clf.best_score_))
best_parametors = gs_clf.best_estimator_.get_params()

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best score: 0.9160849884836253


In [25]:
best_parametors

{'memory': None,
 'steps': [('vect', CountVectorizer(ngram_range=(1, 7))),
  ('tfidf', TfidfTransformer()),
  ('clf', MultinomialNB(alpha=0.008))],
 'verbose': False,
 'vect': CountVectorizer(ngram_range=(1, 7)),
 'tfidf': TfidfTransformer(),
 'clf': MultinomialNB(alpha=0.008),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 7),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': None,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_idf': True,
 'clf__alpha': 0.008,
 'clf__class_prior': None,
 'clf__fit_prior': True}

In [26]:
pred = gs_clf.best_estimator_.predict(valid_data)
# pred = text_clf.predict(valid_data)
np.mean(pred == valid_label)

0.9262233951006827

import pickle
with open("naive_bayes_concat_5gram.pkl", "wb") as f:
    pickle.dump(text_clf, f)

In [27]:
pred = gs_clf.best_estimator_.predict(test["concat"])

In [28]:
sample_submission=pd.read_csv('../DATA/open/sample_submission.csv')
sample_submission['label']=pred
sample_submission.to_csv('../DATA/Result/NaiveBayes.csv', index=False)

[CV] END ..........clf__alpha=0.01, vect__ngram_range=(1, 5); total time=  58.0s
[CV] END ..........clf__alpha=0.01, vect__ngram_range=(1, 7); total time= 1.3min
[CV] END ..........clf__alpha=0.01, vect__ngram_range=(1, 9); total time= 1.7min
[CV] END .........clf__alpha=0.008, vect__ngram_range=(1, 5); total time=  46.0s
[CV] END .........clf__alpha=0.008, vect__ngram_range=(1, 7); total time= 1.3min
[CV] END .........clf__alpha=0.008, vect__ngram_range=(1, 9); total time= 1.7min
[CV] END .........clf__alpha=0.006, vect__ngram_range=(1, 5); total time=  47.7s
[CV] END .........clf__alpha=0.006, vect__ngram_range=(1, 7); total time= 1.3min
[CV] END .........clf__alpha=0.006, vect__ngram_range=(1, 9); total time= 1.7min
[CV] END ..........clf__alpha=0.01, vect__ngram_range=(1, 5); total time=  57.8s
[CV] END ..........clf__alpha=0.01, vect__ngram_range=(1, 7); total time= 1.4min
[CV] END ..........clf__alpha=0.01, vect__ngram_range=(1, 9); total time= 1.8min
[CV] END .........clf__alpha